# CHSolver User Tutorial

## Requirements
We assume the following software will be already available prior to installation:
- GCC/10.2.0
- CMake/3.20.1
- OpenMPI/4.0.5
- HDF5/1.10.7
- Python/3.8.6
- FFTW/3.3.8

## Installation
The repository can be git cloned via the command  $\texttt{ git clone --recursive git@github.com:HetSys/CHSolver.git}$

The code requires some external dependancies, which are linked to the repository through the use of git submodules. The bash script $\texttt{build\_deps}$ is provided in order to build and install these locally, within the repository.

Make is used as the main driver for the executable compilation and support. Supported commands are as follows:
- make: Compile and link the chsolver executable
- make clean: Clean the executables, .o and .mod files
- make logpurge: Clean all .log files from the ./logs dir
- make docs: Generate developer documentation (requires a doxygen executable on path, tested with Doxygen/1.8.17)
- make tests: Compile and link a unit tests executable (requires pFUnit to have been installed within the repository)

## Handling JSON input data
JSON files are used by both the Python API and the $\texttt{chsolver}$ executable in order to more easily handle input parameters to the executable. Though it is possible to use the the executable without requiring a JSON file, it is strongly recommended.

The JSON datastructure operates via nesting of key : value pairs. At the root level, the JSON file is subdivided into "runs", where each run contains the full set of parameters required for proper execution of the program. A simple example file is shown below.

```
{
  "default": {
    "L": 1.0,
    "A": 1.0,
    "M": 0.25,
    "K": 0.0004,
    "p0": -1.0,
    "p1": 1.0,
    "grid_level": 7,
    "grid_type": "r",
    "T": [
      0.0,
      0.05,
      0.1
    ]
  }
}
```

The run, called "default" contains a nested set of parameters ("L", "A", "M", ...) which would all be searched by name during execution of the program. To add alternative runs directly to the file, it is recommended to first copy and paste the default run, rename it, and thn modify the values associated with the parameters.

### Handling the JSON file

## Using the $\texttt{chsolver}$ executable